# Dog vs. Cat Classifier
This notebook is supposed to experiment on different aspects of a image classifier: types of layers (fully connected, convolutional), training sample sizes, image generator, pretrained models, etc. 

## Image Generator tools from _keras_

In [1]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
#from PIL import Image
import time
from keras.utils import np_utils, multi_gpu_model

Using TensorFlow backend.


Prepare the data, using the .flow_from_directory() method. 

In [3]:
batch_size = 32

# this generator is used to read all image into a large numpy array
read_datagen = ImageDataGenerator()

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=20,
        horizontal_flip=True,
        fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# read the images into memory as a large numpy array
batch = 0
start = time.time()
for X_train, y_train in read_datagen.flow_from_directory(
        '/scratch/bnallamo_armis/wsliu/dogs_cats/train',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=23000,
        class_mode='categorical'):  # since we use binary_crossentropy loss, we need binary labels
    batch += 1
    if batch >= 1:
        break  # since we use binary_crossentropy loss, we need binary labels

batch = 0
for X_test, y_test in  read_datagen.flow_from_directory(
        '/scratch/bnallamo_armis/wsliu/dogs_cats/data/train2000',
        target_size=(224, 224),
        batch_size=2000,
        class_mode='categorical'):
    batch += 1
    if batch >= 1:
        break
print("Time used for reading data: ", time.time() - start)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Time used for reading data:  210.5278878211975


In [7]:
# define the flow from the data in the memory
G = 2
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size*G)
validation_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size*G)

## Model specification

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import tensorflow as tf

In [6]:
# build the model on CPU memory:
with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), strides=(2,2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), strides=(2,2)))

    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), strides=(2,2)))

    model.add(Conv2D(256, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), strides=(2,2)))
# the model so far outputs 3D feature maps (height, width, features)

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))

In [11]:
# duplicate model on each GPU
parallel_model = multi_gpu_model(model, gpus=G)

#sgd = SGD(lr=0.5, decay=1e-6, momentum=0.5, nesterov=True)
parallel_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model training:

In [12]:
start = time.time()
parallel_model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // (batch_size*G),
        epochs=5,
        validation_data=validation_generator,
        validation_steps=validation_generator.n // (batch_size*G));
print("Time used for training model: ", time.time()-start)

Epoch 1/5
359/359 [==============================] - 298s 831ms/step - loss: 0.6635 - acc: 0.5895 - val_loss: 0.5868 - val_acc: 0.6724
Epoch 2/5
359/359 [==============================] - 279s 777ms/step - loss: 0.5587 - acc: 0.7082 - val_loss: 0.4721 - val_acc: 0.7802
Epoch 3/5
359/359 [==============================] - 276s 770ms/step - loss: 0.4846 - acc: 0.7701 - val_loss: 0.4183 - val_acc: 0.8216
Epoch 4/5
359/359 [==============================] - 263s 732ms/step - loss: 0.4357 - acc: 0.7978 - val_loss: 0.3466 - val_acc: 0.8538
Epoch 5/5
359/359 [==============================] - 271s 754ms/step - loss: 0.3826 - acc: 0.8289 - val_loss: 0.3046 - val_acc: 0.8674
Time used for training model:  1389.4613313674927


In [6]:
model.save_weights('naive_cnn_full_sample.h5')  # always save your weights after training or during training